# TARRAGONA DATATHON, November 2018
## team_10 activities

*   Marianna Polini: Data Scientist / SQL-Databases
*   Christian Villavicencio: Clinician
*   Eva Machado: Clinician
*   Raquel Carbonell: Clinician
*   Diana Gil: Clinician
*   Laia Labad: Clinician
*   Santiago Frias: Data Scientist / Statistics-Models



## Septic Shock query and stay time representation

In [ ]:
# Importacion Librerias
import numpy
print('numpy Version: ',numpy.__version__)
import os
import pandas
print('Pandas Version: ',pandas.__version__)
import matplotlib.pyplot as pyplot
pyplot.rcParams['figure.figsize']= [20,12]
import matplotlib.patches as patches
import matplotlib.path as path
import tensorflow
print('tensorflow Version: ',tensorflow.__version__)
# Visualización Dataframes
from IPython.display import display, HTML

# Acceso repositorio datos Datathon.
from google.colab import auth
from google.cloud import bigquery

In [ ]:
# Autentificacion BigQuery, generacion resumen verificacion a pegar despues de validacion.
auth.authenticate_user()

In [ ]:
# Hosting solo lectura proyecto datathon-tarragona-2018
# Creado proyecto compartido team_10 asignado en este kernel
project_id='datathon-tarragona-2018'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

In [ ]:
# Asignar configuracion de la ejecucion de sentencias SQL en BigQuery
def run_query(query):
  return pandas.io.gbq.read_gbq(query, project_id=project_id, verbose=True, configuration={'query':{'useLegacySql': True}})

# Crude Data

In [ ]:
consulta = run_query("SELECT patientunitstayid,activeupondischarge,diagnosisoffset FROM [physionet-data:eicu_crd.diagnosis] WHERE LOWER(diagnosisstring) LIKE '%septic shock%' ORDER BY patientunitstayid,activeupondischarge, diagnosisoffset")

In [ ]:
consulta

In [ ]:
# Scatter Plot de correlacion entre eje x, eje y, de ejemplo crude data
consulta.plot(figsize = [20,16], kind='scatter',x='patientunitstayid',y='diagnosisoffset' )

# Data for patients registered on Septic Shock Diag with almost near of 3 days

In [ ]:
consulta = run_query("SELECT patientunitstayid,activeupondischarge,diagnosisoffset FROM [physionet-data:eicu_crd.diagnosis] WHERE LOWER(diagnosisstring) LIKE '%septic shock%' AND diagnosisoffset > 4000 ORDER BY patientunitstayid,activeupondischarge,diagnosisoffset")

In [ ]:
consulta

In [ ]:
# Scatter Plot de correlacion entre eje x, eje y, eliminados pacientes de los que no tenemos datos de tres dias
consulta.plot(figsize = [20,16], kind='scatter',x='patientunitstayid',y='diagnosisoffset' )

# Number of unique id's to profile  load balancing

In [ ]:
consulta = run_query("SELECT patientunitstayid FROM [physionet-data:eicu_crd.diagnosis] WHERE LOWER(diagnosisstring) LIKE '%septic shock%' AND diagnosisoffset > 4000 GROUP BY patientunitstayid  ORDER BY patientunitstayid")


In [ ]:
consulta

# Intake Output Balance events to calculate overload. Crude Data.

In [ ]:
consulta = run_query("SELECT patientunitstayid,nettotal,intakeoutputentryoffset FROM [physionet-data:eicu_crd.intakeoutput] ORDER BY patientunitstayid")

In [ ]:
consulta

In [ ]:
# Scatter Plot exploratorio para detectar eventos con valores imposibles a eliminar.
consulta.plot(figsize = [19,15], kind='scatter',x='patientunitstayid',y='nettotal' )

# Intake Output Balance events to calculate overload. Cleaned Data

In [ ]:
consulta = run_query("SELECT f.patientunitstayid , f.nettotal, f.intakeoutputentryoffset FROM [physionet-data:eicu_crd.intakeoutput] AS f WHERE (f.nettotal < 5000) AND (f.nettotal > -5000) ORDER BY f.patientunitstayid, f.intakeoutputentryoffset") 

In [ ]:
consulta

In [ ]:
# Scatter Plot exploratorio de la topologia del balance de fluidos por estancia del paciente
consulta.plot(figsize = [19,15], kind='scatter',x='f_patientunitstayid',y='f_intakeoutputentryoffset' )